In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, random_split
import os

In [2]:
# Перевірка доступності MPS (Metal Performance Shaders)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [3]:
# Директорія з даними
train_dir = "./seg_train"
test_dir = "./seg_test"

In [4]:
# Перетворення для даних
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [5]:
# Завантаження даних
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_data, val_data = random_split(train_dataset, [train_size, val_size])

test_data = datasets.ImageFolder(root=test_dir, transform=transform)

In [8]:
# Завантажувачі даних
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [9]:
# Класи
target_classes = train_dataset.classes
print("Classes:", target_classes)

Classes: ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [10]:
# Визначення CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 18 * 18, 256)
        self.fc2 = nn.Linear(256, len(target_classes))
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [11]:
# Визначення моделі
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
# Функція навчання
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    best_accuracy = 0.0
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            images = images.to(torch.float32)  # Виправлення для MPS
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_accuracy = 100 * correct / total
        
        # Оцінка на валідаційному наборі
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                images = images.to(torch.float32)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%")
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.to("cpu").state_dict(), "best_model.pth")
            model.to(device)

In [13]:
# Навчання
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)

Epoch 1/10, Loss: 0.9857, Train Acc: 62.15%, Val Acc: 69.75%
Epoch 2/10, Loss: 0.6594, Train Acc: 75.92%, Val Acc: 78.59%
Epoch 3/10, Loss: 0.5104, Train Acc: 81.92%, Val Acc: 83.26%
Epoch 4/10, Loss: 0.3952, Train Acc: 85.45%, Val Acc: 83.22%
Epoch 5/10, Loss: 0.3106, Train Acc: 89.16%, Val Acc: 82.79%
Epoch 6/10, Loss: 0.2365, Train Acc: 91.73%, Val Acc: 83.33%
Epoch 7/10, Loss: 0.1795, Train Acc: 93.71%, Val Acc: 83.01%
Epoch 8/10, Loss: 0.1404, Train Acc: 95.09%, Val Acc: 82.37%
Epoch 9/10, Loss: 0.1267, Train Acc: 95.48%, Val Acc: 82.19%
Epoch 10/10, Loss: 0.0942, Train Acc: 96.86%, Val Acc: 82.37%


In [14]:
# Оцінка на тестових даних
model.load_state_dict(torch.load("best_model.pth", map_location="cpu"))
model.to(device)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        images = images.to(torch.float32)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

/var/folders/wq/wx67703j4057_v32jpcwy4_m0000gn/T/ipykernel_99654/385020295.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.p

Test Accuracy: 83.33%


In [15]:
# Використання Transfer Learning
resnet = models.resnet18(weights="IMAGENET1K_V1")
num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features, len(target_classes))
resnet = resnet.to(device)

optimizer_resnet = optim.Adam(resnet.parameters(), lr=0.0001)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/innasnegurova/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 46.3MB/s]


In [16]:
# Навчання ResNet
train_model(resnet, train_loader, val_loader, criterion, optimizer_resnet, epochs=5)

Epoch 1/5, Loss: 0.3157, Train Acc: 89.12%, Val Acc: 92.45%
Epoch 2/5, Loss: 0.1195, Train Acc: 95.96%, Val Acc: 93.69%
Epoch 3/5, Loss: 0.0621, Train Acc: 98.07%, Val Acc: 91.95%
Epoch 4/5, Loss: 0.0385, Train Acc: 98.85%, Val Acc: 92.27%
Epoch 5/5, Loss: 0.0259, Train Acc: 99.29%, Val Acc: 92.73%


In [17]:
# Оцінка ResNet
resnet.load_state_dict(torch.load("best_model.pth", map_location="cpu"))
resnet.to(device)
resnet.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        images = images.to(torch.float32)
        outputs = resnet(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_accuracy_resnet = 100 * correct / total
print(f"ResNet Test Accuracy: {test_accuracy_resnet:.2f}%")

/var/folders/wq/wx67703j4057_v32jpcwy4_m0000gn/T/ipykernel_99654/1325406127.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet.load_state_dict(torch.load("best_model

ResNet Test Accuracy: 93.10%


## Висновки по навчанню моделей
- ResNet18 краще підходить для цієї задачі, оскільки забезпечує вищу точність і стабільність без ознак перенавчання.
- SimpleCNN швидко перенавчається. Для покращення результатів можна:
    - Зменшити кількість епох (оптимальна точність досягається на 5-й епосі).
    - Змінити швидкість навчання
- Transfer Learning працює ефективніше, ніж навчання з нуля. Якщо важлива продуктивність, краще використовувати ResNet або інші попередньо навчені моделі.

📌 Підсумок: Якщо потрібно найкраще співвідношення точності та продуктивності – ResNet18 є оптимальним вибором! 🚀